In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Data Preparation

In [2]:
df = pd.read_csv('imbd_cleaned.csv')

# Mappa di raggruppamento
category_mapping = {
    'movie': 'Film',
    'tvMovie': 'Film',
    'short': 'Shorts',
    'tvShort': 'Shorts',
    'tvSeries': 'Serie TV',
    'tvMiniSeries': 'Serie TV',
    'tvEpisode': 'Serie TV',
    'tvSpecial': 'Speciale TV',
    'video': 'Video',
    'videoGame': 'Videogame'
}

# Applicare la mappa al dataframe
df['titleType'] = df['titleType'].map(category_mapping)

X = df.select_dtypes(include=['number'])

X = X.values
y = np.array(df['titleType'])

In [3]:
from sklearn.model_selection import train_test_split, cross_val_score 

from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100, stratify=y)

In [5]:
from sklearn.preprocessing import StandardScaler

In [6]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Sklearn Multilayer Perceptron

In [7]:
from sklearn.neural_network import MLPClassifier

In [8]:
hidden_layer_sizes = (100, 200,)

In [9]:
clf = MLPClassifier(random_state=0)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

C:\Users\alban\AppData\Roaming\Python\Python313\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy 0.9261036253166828
F1-score [0.91876342 0.96820193 0.89195631 0.27125506 0.54593573 0.75559534]
              precision    recall  f1-score   support

        Film       0.90      0.93      0.92     12815
    Serie TV       0.97      0.97      0.97     24378
      Shorts       0.88      0.90      0.89      4994
 Speciale TV       0.45      0.19      0.27       344
       Video       0.66      0.47      0.55      1543
   Videogame       0.72      0.80      0.76       529

    accuracy                           0.93     44603
   macro avg       0.76      0.71      0.73     44603
weighted avg       0.92      0.93      0.92     44603



In [ ]:
plt.plot(clf.loss_curve_)
plt.show()

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(128, 64, 32,), alpha=0.1,
                    learning_rate='adaptive',
                    activation='tanh', early_stopping=False,
                    momentum=0.9, random_state=0, max_iter=500)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

In [ ]:
plt.plot(clf.loss_curve_)
plt.show()

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(128, 64, 32,), alpha=0.1, learning_rate='adaptive',
                    activation='tanh', early_stopping=True, momentum=0.9, random_state=0)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

In [ ]:
plt.plot(clf.loss_curve_)
plt.show()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
param_list = {
    'hidden_layer_sizes': [(256, 128, 64,), (128, 64, 32,), (64, 32,)],
    'alpha': [0.1, 0.01, 0.001],
    'activation': ['tanh', 'relu', 'logistic'],
    'solver': ['sgd', 'adam'],
    'early_stopping': [True, False]
}

In [ ]:
random_search = RandomizedSearchCV(
    MLPClassifier(random_state=0, max_iter=1000),
    param_distributions=param_list,
    n_jobs=-1,
    refit=True,
    verbose=2
)

In [ ]:
random_search.fit(X_train, y_train)

In [ ]:
print(random_search.best_params_, random_search.best_score_)

# PyTorch

In [ ]:
# !pip3 install torch
# !pip install torchsummary
# !pip install pytorch-ignite

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchsummary import summary
from torch.utils.data import TensorDataset, DataLoader

from ignite.metrics import Accuracy, Loss
from ignite.engine import Engine, Events, create_supervised_trainer, create_supervised_evaluator

In [ ]:
# creates validation set
X_val, X_new_test, y_val, y_new_test = train_test_split(X_test, y_test, test_size=0.2, random_state=42, stratify=y_test)

In [ ]:
# numpy to tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [ ]:
# creates tensor dataset (can be later loaded)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [ ]:
# creates data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
class CustomModel(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(CustomModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.softmax(self.fc3(x))
        return x

In [ ]:
# define hyperparameters
n_classes = len(np.unique(y_train))
input_size = X_train.shape[1]
hidden_size1 = 128
hidden_size2 = 64
output_size = n_classes

In [ ]:
# instanciates model
model = CustomModel(input_size, hidden_size1, hidden_size2, output_size)

In [ ]:
summary(model, input_size=(input_size,))

In [ ]:
# optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
# ignite trainer
trainer = create_supervised_trainer(model, optimizer, criterion, device)

# validation metrics are ignite metrics
val_metrics = {"accuracy": Accuracy(), "loss": Loss(criterion)}

# ignite train and validation evaluators
train_evaluator = create_supervised_evaluator(model, metrics=val_metrics, device=device)
val_evaluator = create_supervised_evaluator(model, metrics=val_metrics, device=device)

training_history = {'accuracy':[],'loss':[]}
validation_history = {'accuracy':[],'loss':[]}

In [ ]:
# For logging purposes we add a function to be executed at the end of every log_interval-th iteration:
log_interval = 100

@trainer.on(Events.ITERATION_COMPLETED(every=log_interval))
def log_training_loss(engine):
    print(f"Epoch[{engine.state.epoch}], Iter[{engine.state.iteration}] Loss: {engine.state.output:.2f}")

# after a training epoch, computes training and validation metrics (train_eval on train_loader and val_eval on val_loader)
@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    train_evaluator.run(train_loader)
    metrics = train_evaluator.state.metrics
    training_history['accuracy'].append(metrics['accuracy']*100)
    training_history['loss'].append(metrics['loss'])
    print(f"Training Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}")

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    val_evaluator.run(val_loader)
    metrics = val_evaluator.state.metrics
    validation_history['accuracy'].append(metrics['accuracy']*100)
    validation_history['loss'].append(metrics['loss'])
    print(f"Validation Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}")

In [ ]:
trainer.run(train_loader, max_epochs=300)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 3))
axes[0].plot(training_history['accuracy'], label='train')
axes[0].plot(validation_history['accuracy'], label='val')
axes[0].set_xlabel('Epochs')
axes[0].set_ylabel('Accuracy')
axes[0].legend()

axes[1].plot(training_history['loss'], label='train')
axes[1].plot(validation_history['loss'], label='val')
axes[1].set_xlabel('Epochs')
axes[1].set_ylabel('Loss')
axes[1].legend()
fig.tight_layout()

plt.show()

### Early Stopping

In [ ]:
from ignite.handlers import EarlyStopping, ModelCheckpoint
from ignite.contrib.handlers import global_step_from_engine

In [ ]:
# model + optimizer + criterion
model = CustomModel(input_size, hidden_size1, hidden_size2, output_size)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# ignite trainer + evaluators
trainer = create_supervised_trainer(model, optimizer, criterion, device)
val_metrics = {"accuracy": Accuracy(), "loss": Loss(criterion)}
train_evaluator = create_supervised_evaluator(model, metrics=val_metrics, device=device)
val_evaluator = create_supervised_evaluator(model, metrics=val_metrics, device=device)

training_history = {'accuracy':[],'loss':[]}
validation_history = {'accuracy':[],'loss':[]}

# loggers + history appends
@trainer.on(Events.ITERATION_COMPLETED(every=log_interval))
def log_training_loss(engine):
    print(f"Epoch[{engine.state.epoch}], Iter[{engine.state.iteration}] Loss: {engine.state.output:.2f}")

# after a training epoch, computes training and validation metrics (train_eval on train_loader and val_eval on val_loader)
@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    train_evaluator.run(train_loader)
    metrics = train_evaluator.state.metrics
    training_history['accuracy'].append(metrics['accuracy']*100)
    training_history['loss'].append(metrics['loss'])
    print(f"Training Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}")

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    val_evaluator.run(val_loader)
    metrics = val_evaluator.state.metrics
    validation_history['accuracy'].append(metrics['accuracy']*100)
    validation_history['loss'].append(metrics['loss'])
    print(f"Validation Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}")

In [ ]:
# return current value of any metric defined in val_metrics
def score_function(engine):
    return engine.state.metrics["accuracy"]

In [ ]:
# define early stopping and model checkpoint
handler = EarlyStopping(patience=50, score_function=score_function, trainer=trainer)

checkpoint = ModelCheckpoint(
    dirname='models',
    filename_prefix='best_NOREG',
    n_saved=1,
    create_dir=True,
    global_step_transform=global_step_from_engine(trainer) # helps fetch the trainer's state
)

# Attach early stopping and model checkpoint to the trainer
val_evaluator.add_event_handler(Events.EPOCH_COMPLETED, handler)
val_evaluator.add_event_handler(Events.EPOCH_COMPLETED, checkpoint, {'model': model})

In [ ]:
trainer.run(train_loader, max_epochs=300)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 3))
axes[0].plot(training_history['accuracy'], label='train')
axes[0].plot(validation_history['accuracy'], label='val')
axes[0].set_xlabel('Epochs')
axes[0].set_ylabel('Accuracy')
axes[0].legend()

axes[1].plot(training_history['loss'], label='train')
axes[1].plot(validation_history['loss'], label='val')
axes[1].set_xlabel('Epochs')
axes[1].set_ylabel('Loss')
axes[1].legend()
fig.tight_layout()

plt.show()

a gigi piace il pisello grande grande

### GRIDSEARCH

In [ ]:
from ignite.handlers import EarlyStopping, ModelCheckpoint
from ignite.contrib.handlers import global_step_from_engine

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import RandomizedSearchCV
from ignite.engine import State

In [ ]:
class PyTorchClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, input_dim, hidden_layer_sizes, activation, optimizer_class, lr, epochs,
                 device, criterion, batch_size, patience=50):
        self.input_dim = input_dim
        self.hidden_layer_sizes = hidden_layer_sizes
        self.activation = activation
        self.optimizer_class = optimizer_class
        self.lr = lr
        self.epochs = epochs
        self.model = None
        self.optimizer = None
        self.device = device
        self.criterion = criterion
        self.batch_size = batch_size
        self.patience = patience
        self.trainer = None

    def fit(self, X, y):
        print("=======================================================")
        print("Fitting on Parameters:", 
          f"Input Dimension: {self.input_dim},\n"
          f"Hidden Layer Sizes: {self.hidden_layer_sizes},\n"
          f"Activation: {self.activation},\n"
          f"Optimizer Class: {self.optimizer_class},\n"
          f"Learning Rate: {self.lr},\n"
          f"Epochs: {self.epochs},\n"
          f"Device: {self.device},\n"
          f"Criterion: {self.criterion},\n"
          f"Batch Size: {self.batch_size},\n"
          f"Patience: {self.patience}\n"
        )

        self.classes_ = np.unique(y)

        # Initialize model and optimizer
        self.model = PyTorchModel(self.input_dim, self.hidden_layer_sizes, self.activation)
        self.optimizer = self.optimizer_class(self.model.parameters(), lr=self.lr)

    # DataLoader
        tensor_dataset = TensorDataset(torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.long))
        self.train_loader = DataLoader(tensor_dataset, batch_size=self.batch_size, shuffle=False)

    # Ignite Trainer
        self.trainer = create_supervised_trainer(self.model, self.optimizer, self.criterion, self.device)

    # Evaluator per loss media (usata per early stopping)
        self.evaluator = create_supervised_evaluator(
           self.model,
            metrics={'loss': Loss(self.criterion)},
            device=self.device
        )

    # Logger durante il training
        self.trainer.add_event_handler(Events.EPOCH_COMPLETED, self.log_training_results)

        @self.trainer.on(Events.ITERATION_COMPLETED(every=10))
        def log_training_loss(engine):
            print(f"Epoch[{engine.state.epoch}], Iter[{engine.state.iteration}] Loss: {engine.state.output:.4f}")

    # Early stopping: calcolato sulla loss media dell'epoca
        def score_function(engine):
            val_loss = engine.state.metrics['loss']
            return -val_loss  # vogliamo minimizzare la loss

    # Dopo ogni epoca, valuta su tutto il training set (potresti cambiarlo con validation se ne hai uno)
        @self.trainer.on(Events.EPOCH_COMPLETED)
        def run_validation_and_early_stopping(engine):
            self.evaluator.run(self.train_loader)

        early_stopping = EarlyStopping(
            patience=self.patience,
            score_function=score_function,
            trainer=self.trainer,
            min_delta=0.001  # miglioramento minimo considerato valido (opzionale)
        )
        self.evaluator.add_event_handler(Events.COMPLETED, early_stopping)

        # ✅ Salva il numero di epoche effettivamente raggiunto
        @self.trainer.on(Events.COMPLETED)
        def save_final_epoch(engine):
            self.final_epoch_ = engine.state.epoch

    # Reset stato trainer per garantire multi-fit con sklearn
        self.trainer.state = State()
        self.trainer.run(self.train_loader, max_epochs=self.epochs)

    def predict(self, X):
        self.model.eval()
        data_loader = DataLoader(TensorDataset(torch.tensor(X, dtype=torch.float32)),
                                 batch_size=self.batch_size, shuffle=False)

        predictions = []
        with torch.no_grad():
            for inputs in data_loader:
                outputs = self.model(inputs[0].to(self.device))
                _, predicted = torch.max(outputs, 1)
                predictions.extend(predicted.cpu().tolist())

        return np.array(predictions)

    def log_training_results(self, trainer):
        self.model.eval()
        total_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, targets in self.train_loader:
                inputs, targets = inputs.to(self.device), targets.to(self.device)
                outputs = self.model(inputs)
                loss = self.criterion(outputs, targets)
                total_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()

        accuracy = 100 * correct / total
        avg_loss = total_loss / len(self.train_loader)
        print(f"Training Results - Epoch[{trainer.state.epoch}] Accuracy: {accuracy:.2f}% - Loss: {avg_loss:.4f}")

    def score(self, X, y):
        y_pred = self.predict(X)
        return (y_pred == y).mean()


In [ ]:
param_grid = {
    'hidden_layer_sizes': [(128, 64, 32), (64, 32, 16), (256, 128, 64)],
    'activation': [nn.ReLU(), nn.Tanh()],
    'lr': [0.001, 0.01, 0.1, 1],
    'optimizer_class': [optim.Adam, optim.SGD],
    # 'epochs': [10, 50, 100, 200],
    'batch_size': [32, 64, 128]
}

In [ ]:
clf = PyTorchClassifier(
    input_dim = input_size,
    hidden_layer_sizes = (128, 64),
    activation = nn.ReLU(),
    optimizer_class = optim.Adam,
    lr = 0.001,
    epochs = 300,
    device = device,
    # metrics = {"accuracy": Accuracy(), "loss": Loss(criterion)},
    criterion = nn.CrossEntropyLoss(),
    batch_size = 32,
    patience=50
)

In [ ]:
# perform randomized grid search
rs = RandomizedSearchCV(clf, param_grid, scoring='accuracy', verbose=True, n_iter=15, cv=3, error_score='raise')
rs.fit(X_train, y_train)

best_model = rs.best_estimator_

print(f"Epoche effettive trovate con early stopping: {best_model.final_epoch_}")


# Prendi i migliori parametri
best_params = rs.best_params_

# Aggiungi manualmente i parametri mancanti
best_params["input_dim"] = X_train.shape[1]
best_params["device"] = "cuda" if torch.cuda.is_available() else "cpu"
best_params["criterion"] = torch.nn.CrossEntropyLoss()

# Aggiorna il numero di epoche con quello trovato da early stopping
best_params["epochs"] = best_model.final_epoch_

# Ricrea il modello finale
final_model = PyTorchClassifier(**best_params)

# Fit completo
final_model.fit(X_train, y_train)


Fitting 3 folds for each of 15 candidates, totalling 45 fits
Fitting on Parameters: Input Dimension: 30,
Hidden Layer Sizes: (64, 32, 16),
Activation: ReLU(),
Optimizer Class: <class 'torch.optim.adam.Adam'>,
Learning Rate: 0.001,
Epochs: 300,
Device: cpu,
Criterion: CrossEntropyLoss(),
Batch Size: 64,
Patience: 50

Training Results - Epoch[1] Accuracy: 90.94% - Loss: 0.6703
Epoch[2], Iter[10] Loss: 0.6677
Training Results - Epoch[2] Accuracy: 92.45% - Loss: 0.6506
Training Results - Epoch[3] Accuracy: 92.83% - Loss: 0.6223
Epoch[4], Iter[20] Loss: 0.6097
Training Results - Epoch[4] Accuracy: 92.45% - Loss: 0.5846
Training Results - Epoch[5] Accuracy: 92.83% - Loss: 0.5416
Epoch[6], Iter[30] Loss: 0.5236
Training Results - Epoch[6] Accuracy: 92.83% - Loss: 0.4984
Training Results - Epoch[7] Accuracy: 93.21% - Loss: 0.4598
Epoch[8], Iter[40] Loss: 0.4521
Training Results - Epoch[8] Accuracy: 94.34% - Loss: 0.4284
Training Results - Epoch[9] Accuracy: 95.09% - Loss: 0.4041
Epoch[10], Ite

2025-04-14 18:49:35,130 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training


Epoch[76], Iter[380] Loss: 0.3134
Training Results - Epoch[76] Accuracy: 99.62% - Loss: 0.3167
Training Results - Epoch[77] Accuracy: 99.62% - Loss: 0.3166
Fitting on Parameters: Input Dimension: 30,
Hidden Layer Sizes: (64, 32, 16),
Activation: ReLU(),
Optimizer Class: <class 'torch.optim.adam.Adam'>,
Learning Rate: 0.001,
Epochs: 300,
Device: cpu,
Criterion: CrossEntropyLoss(),
Batch Size: 64,
Patience: 50

Training Results - Epoch[1] Accuracy: 48.30% - Loss: 0.6831
Epoch[2], Iter[10] Loss: 0.6704
Training Results - Epoch[2] Accuracy: 82.64% - Loss: 0.6709
Training Results - Epoch[3] Accuracy: 90.19% - Loss: 0.6551
Epoch[4], Iter[20] Loss: 0.6396
Training Results - Epoch[4] Accuracy: 90.94% - Loss: 0.6344
Training Results - Epoch[5] Accuracy: 91.70% - Loss: 0.6074
Epoch[6], Iter[30] Loss: 0.5832
Training Results - Epoch[6] Accuracy: 92.08% - Loss: 0.5731
Training Results - Epoch[7] Accuracy: 91.70% - Loss: 0.5327
Epoch[8], Iter[40] Loss: 0.5065
Training Results - Epoch[8] Accuracy: 9

2025-04-14 18:49:36,807 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training


Training Results - Epoch[65] Accuracy: 98.49% - Loss: 0.3262
Epoch[66], Iter[330] Loss: 0.3135
Training Results - Epoch[66] Accuracy: 98.49% - Loss: 0.3262
Training Results - Epoch[67] Accuracy: 98.49% - Loss: 0.3262
Epoch[68], Iter[340] Loss: 0.3135
Training Results - Epoch[68] Accuracy: 98.49% - Loss: 0.3262
Training Results - Epoch[69] Accuracy: 98.49% - Loss: 0.3261
Epoch[70], Iter[350] Loss: 0.3135
Training Results - Epoch[70] Accuracy: 98.49% - Loss: 0.3261
Training Results - Epoch[71] Accuracy: 98.49% - Loss: 0.3261
Epoch[72], Iter[360] Loss: 0.3135
Training Results - Epoch[72] Accuracy: 98.49% - Loss: 0.3261
Fitting on Parameters: Input Dimension: 30,
Hidden Layer Sizes: (64, 32, 16),
Activation: ReLU(),
Optimizer Class: <class 'torch.optim.adam.Adam'>,
Learning Rate: 0.001,
Epochs: 300,
Device: cpu,
Criterion: CrossEntropyLoss(),
Batch Size: 64,
Patience: 50

Training Results - Epoch[1] Accuracy: 34.21% - Loss: 0.7098
Epoch[2], Iter[10] Loss: 0.6776
Training Results - Epoch[2]

...

Training Results - Epoch[43] Accuracy: 61.51% - Loss: 0.7466
Epoch[44], Iter[220] Loss: 0.9799
Training Results - Epoch[44] Accuracy: 61.51% - Loss: 0.7466
Training Results - Epoch[45] Accuracy: 61.51% - Loss: 0.7466
Epoch[46], Iter[230] Loss: 0.9799
Training Results - Epoch[46] Accuracy: 61.51% - Loss: 0.7466
Training Results - Epoch[47] Accuracy: 61.51% - Loss: 0.7466
Epoch[48], Iter[240] Loss: 0.9799
Training Results - Epoch[48] Accuracy: 61.51% - Loss: 0.7466
Training Results - Epoch[49] Accuracy: 61.51% - Loss: 0.7466
Epoch[50], Iter[250] Loss: 0.9799
Training Results - Epoch[50] Accuracy: 61.51% - Loss: 0.7466
Training Results - Epoch[51] Accuracy: 61.51% - Loss: 0.7466
Fitting on Parameters: Input Dimension: 30,
Hidden Layer Sizes: (256, 128, 64),
Activation: ReLU(),
Optimizer Class: <class 'torch.optim.adam.Adam'>,
Learning Rate: 0.1,
Epochs: 300,
Device: cpu,
Criterion: CrossEntropyLoss(),
Batch Size: 64,
Patience: 50

Training Results - Epoch[1] Accuracy: 34.21% - Loss: 0.961

2025-04-14 18:51:00,506 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training


Training Results - Epoch[50] Accuracy: 34.21% - Loss: 0.9614
Training Results - Epoch[51] Accuracy: 34.21% - Loss: 0.9614
Fitting on Parameters: Input Dimension: 30,
Hidden Layer Sizes: (256, 128, 64),
Activation: Tanh(),
Optimizer Class: <class 'torch.optim.sgd.SGD'>,
Learning Rate: 0.1,
Epochs: 300,
Device: cpu,
Criterion: CrossEntropyLoss(),
Batch Size: 32,
Patience: 50

Epoch[1], Iter[10] Loss: 0.4863
Training Results - Epoch[1] Accuracy: 94.72% - Loss: 0.4469
Epoch[2], Iter[20] Loss: 0.4069
Training Results - Epoch[2] Accuracy: 95.98% - Loss: 0.3871
Epoch[3], Iter[30] Loss: 0.4219
Training Results - Epoch[3] Accuracy: 97.49% - Loss: 0.3667
Epoch[4], Iter[40] Loss: 0.3394
Epoch[4], Iter[50] Loss: 0.3685
Training Results - Epoch[4] Accuracy: 97.49% - Loss: 0.3566
Epoch[5], Iter[60] Loss: 0.3227
Training Results - Epoch[5] Accuracy: 97.99% - Loss: 0.3507
Epoch[6], Iter[70] Loss: 0.3328
Training Results - Epoch[6] Accuracy: 97.99% - Loss: 0.3467
Epoch[7], Iter[80] Loss: 0.3676
Epoch[7

2025-04-14 18:51:03,192 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training


Training Results - Epoch[59] Accuracy: 99.25% - Loss: 0.3232
Epoch[60], Iter[770] Loss: 0.3147
Epoch[60], Iter[780] Loss: 0.3203
Training Results - Epoch[60] Accuracy: 99.25% - Loss: 0.3232
Epoch[61], Iter[790] Loss: 0.3162
Training Results - Epoch[61] Accuracy: 99.25% - Loss: 0.3231
Epoche effettive trovate con early stopping: 61
Fitting on Parameters: Input Dimension: 30,
Hidden Layer Sizes: (256, 128, 64),
Activation: Tanh(),
Optimizer Class: <class 'torch.optim.sgd.SGD'>,
Learning Rate: 0.1,
Epochs: 61,
Device: cpu,
Criterion: CrossEntropyLoss(),
Batch Size: 32,
Patience: 50

Epoch[1], Iter[10] Loss: 0.4722
Training Results - Epoch[1] Accuracy: 93.47% - Loss: 0.4352
Epoch[2], Iter[20] Loss: 0.4048
Training Results - Epoch[2] Accuracy: 95.73% - Loss: 0.3881
Epoch[3], Iter[30] Loss: 0.4184
Training Results - Epoch[3] Accuracy: 96.48% - Loss: 0.3693
Epoch[4], Iter[40] Loss: 0.3431
Epoch[4], Iter[50] Loss: 0.3761
Training Results - Epoch[4] Accuracy: 97.24% - Loss: 0.3588
Epoch[5], Ite

In [ ]:
print(rs.best_score_, rs.best_params_)

0.9798929141034405 {'optimizer_class': <class 'torch.optim.sgd.SGD'>, 'lr': 0.1, 'hidden_layer_sizes': (256, 128, 64), 'batch_size': 32, 'activation': Tanh(), 'input_dim': 30, 'device': 'cpu', 'criterion': CrossEntropyLoss(), 'epochs': 61}


In [ ]:
y_pred = final_model.predict(X_test).astype(int)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.9707602339181286
F1-score [0.96062992 0.97674419]
              precision    recall  f1-score   support

           0       0.97      0.95      0.96        64
           1       0.97      0.98      0.98       107

    accuracy                           0.97       171
   macro avg       0.97      0.97      0.97       171
weighted avg       0.97      0.97      0.97       171



In [ ]:
final_model

PyTorchClassifier(activation=Tanh(), batch_size=32,
                  criterion=CrossEntropyLoss(), device='cpu', epochs=61,
                  hidden_layer_sizes=(256, 128, 64), input_dim=30, lr=0.1,
                  optimizer_class=<class 'torch.optim.sgd.SGD'>)